In [ ]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
import os
import re
import csv
import time
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date
PTT_URL = 'https://www.ptt.cc'
target_url = '/bbs/nb-shopping/search?page=1&q=surface%20pro%203'
csv_filename = 'ptt_surface3.csv'


fileexist = os.path.isfile(csv_filename)
header = ['category', 'title', 'href','push_count','author','price','date']

with open(csv_filename,'w',newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    #writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    #writer.writeheader()

def word_in_string(pre_str, word):
    flag = 0
    for i in range(len(pre_str)-len(word)+1):
        if pre_str[i:i+len(word)]==word:
            #print(pre_str[i:i+len(word)], word)
            flag=1
            break
    return flag
    
def digit_filter_and_convert(pre_str):
    new_str = ''
    new_int = 1
    digit_str='0123456789'
    for s in pre_str:
        for i in digit_str:
            if s==i:
                new_str = new_str+i
                break
    new_int=int(new_str)
    return new_int


def get_date_from_content(content_all):
    month_lib = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    #post_date = -1
    #post_date = date(year = 2020, month = 1, day = 1)
    yy,mm,dd,HH,MM,SS=1,1,1,1,1,1
    post_datetime = -1
    for index in range(len(content_all)-1):
            if content_all[index:index+2] =="時間":
                time_str = content_all[index+2:index+27]
                time_list = re.split(':| |\n',time_str)
                while '' in time_list:
                    time_list.remove('')
                #print(time_list)
                for m in range(len(month_lib)):
                    if month_lib[m] == time_list[1]:
                        mm = m+1
                        try:
                            yy = digit_filter_and_convert(time_list[6])
                            dd = digit_filter_and_convert(time_list[2])
                            HH = digit_filter_and_convert(time_list[3])
                            MM = digit_filter_and_convert(time_list[4])
                            SS = digit_filter_and_convert(time_list[5])
                            post_datetime = datetime(year = yy, month = m+1, day = dd, hour = HH, minute = MM, second = SS)
                        except:
                            pass
                #print(time_str)
                print(post_datetime)
                break       
    return post_datetime 

def get_web_page(url):
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

def get_content(dom):
    soup = BeautifulSoup(dom, 'html.parser') 
    divs = soup.find_all('div', id = 'main-container')
    price = "none"
    for d in divs:
        content=re.split('：| |\n',d.text)
        post_date = get_date_from_content(d.text)
        #print(d.text)
        #print(content)
        for index in range(len(content)):
            if word_in_string(content[index], '交易價格'):
                pre_price = content[index+1]
                price_list = filter(str.isdigit, pre_price)
                price = ''.join(list(price_list))
    return price, post_date

def get_articles(dom):
    try :
        soup = BeautifulSoup(dom, 'html.parser') 
        paging_div = soup.find('div', 'btn-group btn-group-paging')# 取得上一頁的連結
        prev_url = paging_div.find_all('a')[1]['href']
        articles = []  # 儲存取得的文章資料
        divs = soup.find_all('div', 'r-ent')
        for d in divs:
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                #D_day = d.find('div', 'date').text.strip()
                href = d.find('a')['href']
                price, post_date = get_content(get_web_page(PTT_URL + href))
                title = d.find('a').text
                author = d.find('div', 'author').text if d.find('div', 'author') else ''
                titles = title.split("]") 
                #print(titles)
                if len(titles)==2:
                    articles.append({
                        'category':titles[0][1:3],
                        'title': titles[1][1:],
                        'href': href,
                        'author': author,
                        'price' : price,
                        'date' : post_date
                    })
        with open(csv_filename,'a',newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            for article in articles:
                #print(article)
                writer.writerow(article)
        return articles, prev_url
    except:
        return 1, -1

    
if __name__ == '__main__':
    
    '''
    articles,detail = [],[]
    new_articles = 1
    current_page = get_web_page(PTT_URL + target_url)
    if current_page:
        new_articles = get_articles(current_page)
        articles += new_articles
    '''
    
    articles,detail = [],[]
    new_articles = 1
    while new_articles:
        if new_articles!=1 :
            articles += new_articles
        try:
            current_page = get_web_page(PTT_URL + target_url)
        except:
            current_page = -1
        if current_page:
            new_articles, target_url = get_articles(current_page)
    '''
    with open(csv_filename,'w',newline='') as csvfile:
        writer = csv.writer(csvfile)
        fieldnames = ['category', 'title', 'href','author','price','date']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for article in articles:
            print(article)
            writer.writerow(article)
    
    #for a in articles:
        #print(a)
    '''


2019-09-07 23:36:31
2019-09-02 08:29:49
2019-08-26 14:39:42
2019-08-22 15:23:34
2019-08-22 07:19:52
2019-08-12 15:21:40
2019-07-26 22:40:20
2019-07-22 21:08:25
2019-07-14 17:38:28
2019-07-12 19:28:37
2019-06-24 19:26:31
2019-06-18 12:40:56
2019-06-18 09:03:40
2019-06-14 11:30:42
2019-06-04 18:51:25
2019-05-30 21:05:30
2019-05-28 13:15:23
2019-04-16 16:17:58
2019-04-04 20:44:35
